In [0]:
from pyspark.sql.types import StructType, StringType, BooleanType, IntegerType, DecimalType, DateType, NullType, StructField
from pyspark.sql.functions import col, when, cast, date_format, year, month, regexp_extract, regexp_replace, count, desc, countDistinct, max, min, sum

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import SparkSession
spark =  SparkSession.builder.appName("Paris Olympic Data Analysis").getOrCreate()

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS my_olympics_db;


In [0]:
%fs
databricks fs cp "/home/oem/Desktop/DataEngineering/Datasets/Athletes.csv" dbfs:/FileStore/tables/Athletes.csv  
#cant do it  as we need to first load the data ito the dbfs 


In [0]:
df_medals = spark.read.option("header", "true").option("inferSchema","true").csv("/FileStore/tables/Medals.csv")  
# tranformation as spark is lazy evaluation doesnt get executed

In [0]:
df_medals.show()   # runs now  ---- as this is an action
df_medals.printSchema()   # change columns as these are not string values  #inferschema is good but automatically converts bool to int too  
# hence create own schema using structtype

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
medals_schema = StructType([StructField("Rank", IntegerType(), True),
                            StructField("TeamCountry", StringType(), True),
                            StructField("Gold", IntegerType(), True),
                            StructField("Silver", IntegerType(), True),
                            StructField("Bronze", IntegerType(), True),
                            StructField("Total", IntegerType(), True),
                            StructField("Rank By Total", IntegerType(), True)                            
                            ])

In [0]:
df_medals_schema = spark.read.option("header", "true").option("inferSchema","true").schema(medals_schema).csv("/FileStore/tables/Medals.csv")  

In [0]:
df_medals_schema.printSchema()
display(df_medals_schema.show(3))

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank By Total: integer (nullable = true)

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank By Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
+----+--------------------+----+------+------+-----+-------------+
only showing top 3 rows



In [0]:
%sql
SELECT * FROM Medals
LIMIT 10;


In [0]:
df_athletes = spark.read.option("header", "true").option("inferSchema","true").csv("/FileStore/tables/Athletes.csv")  

In [0]:
df_athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
athletes_schema = StructType([
    StructField("PersonName", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Discipline", StringType(), True)
])

In [0]:
%fs ls /FileStore/tables/


In [0]:
df_athletes_schema = spark.read.option("header", "true").schema(athletes_schema).csv("/FileStore/tables/Athletes.csv")  

In [0]:
df_athletes_schema.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
df_athletes_schema.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
%fs ls /FileStore/tables/


In [0]:
%fs ls /FileStore/tables/Coaches.csv


In [0]:
df_coaches = spark.read.option("header", "true").csv("/FileStore/tables/Coaches.csv")  

In [0]:
df_coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
df_coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [0]:
coaches_schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Discipline", StringType(), True),
    StructField("Event", StringType(), True)
])

In [0]:
df_coaches_schema = spark.read.option("header", "true").schema(coaches_schema).csv("/FileStore/tables/Coaches.csv")  

In [0]:
df_coaches_schema.printSchema()
df_coaches_schema.show()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleybal

In [0]:
df_entriesgender = spark.read.option("header", "true").csv("/FileStore/tables/EntriesGender.csv")  

In [0]:
df_entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
entries_gender_schema = StructType([
    StructField("Discipline", StringType(), True),
    StructField("Female", IntegerType(), True),
    StructField("Male", IntegerType(), True),
    StructField("Total", IntegerType(), True)
])

In [0]:
%fs ls /FileStore/tables/


In [0]:
df_entriesgender_schema = spark.read.option("header", "true").schema(entries_gender_schema).csv("/FileStore/tables/EntriesGender.csv")  

In [0]:
df_entriesgender_schema.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
df_entriesgender_schema.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
df_teams = spark.read.option("header","true").csv("/FileStore/tables/Teams.csv")  

In [0]:

teams_schema = StructType([
    StructField("TeamName", StringType(), True),
    StructField("Discipline", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Event", StringType(), True)
])

In [0]:
df_teams_schema= spark.read.option("header","true").schema(teams_schema).csv("/FileStore/tables/Teams.csv")  

In [0]:
df_teams_schema.printSchema()
df_teams_schema.show()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|     

In [0]:
team_df = spark.read.schema(teams_schema).format("CSV").option("header","true").load("/FileStore/tables/Teams.csv")

In [0]:
team_df.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
%fs ls /FileStore/tables/


path,name,size,modificationTime
dbfs:/FileStore/tables/Athletes.csv,Athletes.csv,418492,1750216932000
dbfs:/FileStore/tables/Coaches.csv,Coaches.csv,16889,1750216930000
dbfs:/FileStore/tables/EntriesGender.csv,EntriesGender.csv,1123,1750216930000
dbfs:/FileStore/tables/Medals.csv,Medals.csv,2414,1750216930000
dbfs:/FileStore/tables/Teams.csv,Teams.csv,35270,1750216929000
dbfs:/FileStore/tables/retail_sales_dataset/,retail_sales_dataset/,0,0
dbfs:/FileStore/tables/retail_sales_pipeline/,retail_sales_pipeline/,0,0


In [0]:
# Delete duplicate/extra files
dbutils.fs.rm("dbfs:/FileStore/tables/Coaches-1.csv", True)
dbutils.fs.rm("dbfs:/FileStore/tables/EntriesGender-1.csv", True)
dbutils.fs.rm("dbfs:/FileStore/tables/EntriesGender-2.csv", True)
dbutils.fs.rm("dbfs:/FileStore/tables/Medals-1.csv", True)
dbutils.fs.rm("dbfs:/FileStore/tables/Teams-1.csv", True)


In [0]:
%sql
SELECT * FROM hive_metastore.my_olympics_db.medals
LIMIT 10;


In [0]:
Find distinct countries that participated.

Count the number of athletes per country.

List the top 5 countries with the most athletes.

Group athletes by discipline and count how many per discipline.

Find athletes who participated in more than one discipline.

Filter athletes who belong to “India”.

In [0]:
#Find distinct countries that participated.
df_athletes_schema.columns

Out[38]: ['PersonName', 'Country', 'Discipline']

In [0]:
df_distinct_countries = df_athletes_schema.select("Country").distinct()
df_distinct_countries.show()

+--------------------+
|             Country|
+--------------------+
|Islamic Republic ...|
|            Malaysia|
|           Singapore|
|             Germany|
|            Maldives|
|               Sudan|
|              France|
|             Belgium|
|               Qatar|
|               Chile|
|               Italy|
|              Norway|
|               Spain|
|United States of ...|
|           Indonesia|
|          Azerbaijan|
|        Saudi Arabia|
|          Uzbekistan|
|           Australia|
|               Egypt|
+--------------------+
only showing top 20 rows



In [0]:
#Count the number of athletes per country.
df_no_players_country = df_athletes_schema.groupBy("Country").agg(count(col("PersonName")).alias("Total No of Athletes per country")).orderBy(desc("Total No of Athletes per country"))

In [0]:
df_no_players_country.show()

+--------------------+--------------------------------+
|             Country|Total No of Athletes per country|
+--------------------+--------------------------------+
|United States of ...|                             615|
|               Japan|                             586|
|           Australia|                             470|
|People's Republic...|                             401|
|             Germany|                             400|
|              France|                             377|
|              Canada|                             368|
|       Great Britain|                             366|
|               Italy|                             356|
|               Spain|                             324|
|                 ROC|                             318|
|              Brazil|                             291|
|         Netherlands|                             274|
|   Republic of Korea|                             223|
|         New Zealand|                          

In [0]:
#List the top 5 countries with the most athletes.
df_no_players_country_top_five = df_athletes_schema.groupBy("Country").agg(count(col("PersonName")).alias("Total No of Athletes per country")).orderBy(desc("Total No of Athletes per country")).show(5)

+--------------------+--------------------------------+
|             Country|Total No of Athletes per country|
+--------------------+--------------------------------+
|United States of ...|                             615|
|               Japan|                             586|
|           Australia|                             470|
|People's Republic...|                             401|
|             Germany|                             400|
+--------------------+--------------------------------+
only showing top 5 rows



In [0]:
#List the top 5 countries with the most athletes.
df_no_players_country_top_five = df_athletes_schema.groupBy("Country").agg(count(col("PersonName")).alias("Total No of Athletes per country")).orderBy(desc("Total No of Athletes per country")).limit(5)

In [0]:
df_no_players_country_top_five.show()

+--------------------+--------------------------------+
|             Country|Total No of Athletes per country|
+--------------------+--------------------------------+
|United States of ...|                             615|
|               Japan|                             586|
|           Australia|                             470|
|People's Republic...|                             401|
|             Germany|                             400|
+--------------------+--------------------------------+



In [0]:
#Group athletes by discipline and count how many per discipline.
athletes_by_discipline_count = df_athletes_schema.groupBy("Discipline")\
    .agg(count(col("PersonName"))\
        .alias("Count_People_Discipline"))\
            .orderBy("Count_People_Discipline")

In [0]:
athletes_by_discipline_count.show()

+--------------------+-----------------------+
|          Discipline|Count_People_Discipline|
+--------------------+-----------------------+
|Cycling BMX Frees...|                     19|
|Trampoline Gymnas...|                     31|
|      Sport Climbing|                     37|
|             Surfing|                     38|
|  Cycling BMX Racing|                     43|
|   Marathon Swimming|                     49|
|      3x3 Basketball|                     62|
|   Modern Pentathlon|                     69|
|Cycling Mountain ...|                     74|
|              Karate|                     77|
|       Skateboarding|                     77|
|        Canoe Slalom|                     78|
|    Beach Volleyball|                     90|
| Rhythmic Gymnastics|                     95|
|   Artistic Swimming|                     98|
|           Triathlon|                    106|
|                Golf|                    115|
|             Archery|                    122|
|           T

In [0]:
#Find athletes who participated in more than one discipline.
athletes_more_than_one_discipline_df = df_athletes_schema.groupBy("PersonName")\
    .agg(count("Discipline").alias("discipline_count"))\
    .filter(col("discipline_count")> 1)

In [0]:
athletes_more_than_one_discipline_df.show()

+--------------------+----------------+
|          PersonName|discipline_count|
+--------------------+----------------+
|         PEREZ Paola|               2|
|         HAVIK Yoeri|               2|
|         ALI Mohamed|               2|
|       GANNA Filippo|               2|
|       WATANABE Yuta|               2|
|       ALVAREZ Jorge|               2|
|van ROUWENDAAL Sh...|               2|
|         PEREZ Maria|               2|
|      PORTELA Teresa|               2|
|PALTRINIERI Gregorio|               2|
|         KIM Hyunsoo|               2|
|           WANG Yang|               2|
|   WELLBROCK Florian|               2|
|          HALL James|               2|
|       MARTIN Daniel|               2|
|       KOPECKY Lotte|               2|
|        DYGERT Chloe|               2|
|     KURBANOV Ruslan|               2|
|          SUN Jiajun|               2|
|             LI Qian|               2|
+--------------------+----------------+
only showing top 20 rows



In [0]:
#Filter athletes who belong to “India”
athletes_India_df = df_athletes_schema.select("PersonName","Country")\
    .filter(col("Country") == "India")

In [0]:
athletes_India_df.show()

+--------------------+-------+
|          PersonName|Country|
+--------------------+-------+
|       ACHANTA Kamal|  India|
|                AMIT|  India|
|         ANSHU Anshu|  India|
|         ANTONY Alex|  India|
|         ASHOK Aditi|  India|
|     B. Sai Praneeth|  India|
|BAATTH Simranjit ...|  India|
|     BAJRANG Bajrang|  India|
|BAJWA Angad Vir S...|  India|
|        BATRA Manika|  India|
|         BHAKER Manu|  India|
|     BHAMBRI Sarthak|  India|
|   BORGOHAIN Lovlina|  India|
|CHADALAVADA ANAND...|  India|
|         CHAND Dutee|  India|
|     CHANDELA Apurvi|  India|
|       CHOPRA Neeraj|  India|
|           DAS Atanu|  India|
|DESWAL Yashaswini...|  India|
|       DEVI Sharmila|  India|
+--------------------+-------+
only showing top 20 rows



In [0]:
df_coaches_schema.columns

Out[105]: ['Name', 'Country', 'Discipline', 'Event']

In [0]:
                                                            Find coaches who participated in more than one event.

                                                            Count number of coaches per country.

                                                                List all disciplines coached by “India”.

                                                                Find total number of unique coaches.

                                                                Identify coaches who coached multiple disciplines.

                                                                Find top 5 countries with most coaches.

In [0]:
# Coaches.csv

#Count number of coaches per country.
coaches_country_df = df_coaches_schema.groupBy("Country")\
    .agg(count(col("Name")).alias("No_of_coaches"))\
        .orderBy(desc("No_of_coaches"))


In [0]:
coaches_country_df.show()

+--------------------+-------------+
|             Country|No_of_coaches|
+--------------------+-------------+
|               Japan|           35|
|               Spain|           28|
|United States of ...|           28|
|           Australia|           22|
|              Canada|           16|
|               Italy|           14|
|People's Republic...|           12|
|               Egypt|           12|
|        South Africa|           12|
|                 ROC|           12|
|           Argentina|           11|
|              France|           10|
|           Venezuela|           10|
|         Netherlands|           10|
|             Nigeria|            9|
|             Germany|            9|
|              Mexico|            8|
|         New Zealand|            8|
|       Great Britain|            7|
|   Republic of Korea|            7|
+--------------------+-------------+
only showing top 20 rows



---------------------------------------------------------------------------
AnalysisException Traceback (most recent call last)
File :3
 1 #Find coaches who participated in more than one event.
----> 3 coaches_event_gt_one = df_coaches_schema.groupBy("Name")\
 4 .agg(count(col("Event"))).alias("Count_Event")\
 5 .filter(col("Count_Event")> 1)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function. .wrapper(*args, **kwargs)
 46 start = time.perf_counter()
 47 try:
---> 48 res = func(*args, **kwargs)
 49 logger.log_success(
 50 module_name, class_name, function_name, time.perf_counter() - start, signature
 51 )
 52 return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3125, in DataFrame.filter(self, condition)
 3123 jdf = self._jdf.filter(condition)
 3124 elif isinstance(condition, Column):
-> 3125 jdf = self._jdf.filter(condition._jc)
 3126 else:
 3127 raise PySparkTypeError(
 3128 error_class="NOT_COLUMN_OR_STRING",
 3129 message_parameters={"arg_name": "condition", "arg_type": type(condition).__name__},
 3130 )

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
 1315 command = proto.CALL_COMMAND_NAME +\
 1316 self.command_header +\
 1317 args_command +\
 1318 proto.END_COMMAND_PART
 1320 answer = self.gateway_client.send_command(command)
-> 1321 return_value = get_return_value(
 1322 answer, self.gateway_client, self.target_id, self.name)
 1324 for temp_arg in temp_args:
 1325 temp_arg._detach()

File /databricks/spark/python/pyspark/errors/exceptions.py:234, in capture_sql_exception. .deco(*a, **kw)
 230 converted = convert_exception(e.java_exception)
 231 if not isinstance(converted, UnknownException):
 232 # Hide where the exception came from that shows a non-Pythonic
 233 # JVM exception message.
--> 234 raise converted from None
 235 else:
 236 raise

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Count_Event` cannot be resolved. Did you mean one of the following? [`Count_Event`.`Name`, `Count_Event`.`count(Event)`].;
'Filter ('Count_Event > 1)
+- SubqueryAlias Count_Event
 +- Aggregate [Name#1314], [Name#1314, count(Event#1317) AS count(Event)#1536L]
 +- Relation [Name#1314,Country#1315,Discipline#1316,Event#1317] csv

In [0]:
#Find coaches who participated in more than one event.
coaches_event_gt_one = df_coaches_schema.groupBy("Name") \
    .agg(count("Event").alias("Count_Event")) \
    .filter(col("Count_Event") > 1)

In [0]:
coaches_event_gt_one.show()

+--------------------+-----------+
|                Name|Count_Event|
+--------------------+-----------+
|GIALLOMBARDO Patr...|          2|
|       FUJIKI Mayuko|          2|
|       SZAUDER Gabor|          2|
|       ELAFANDI Nour|          2|
|REQUENA PERICAS J...|          2|
|   FARINELLI Roberta|          2|
|    MONTICO Loredana|          2|
|            WANG Jie|          2|
|    GUERRERO Rolando|          2|
|   CHEPAK Anastasiya|          2|
| RETEGUI Carlos Jose|          2|
|    TARRES CAMPA Ana|          2|
|  CHERNETSKA Natalia|          2|
+--------------------+-----------+



In [0]:
# List all disciplines coached by “India”.
coaches_disciplined_india_df = df_coaches_schema.select("Discipline", "Country")\
    .filter(col("Country")=="India")\
        .distinct()

In [0]:
coaches_disciplined_india_df.show()

+----------+-------+
|Discipline|Country|
+----------+-------+
|    Hockey|  India|
+----------+-------+



In [0]:
#         Find total number of unique coaches.

no_unique_coaches = df_coaches_schema.select("Name").distinct().agg(count("Name").alias("Total_Count_of_unique_coaches"))
        

In [0]:

no_unique_coaches.show()


+-----------------------------+
|Total_Count_of_unique_coaches|
+-----------------------------+
|                          381|
+-----------------------------+



In [0]:
no_unique_coaches = df_coaches_schema.agg(countDistinct("Name").alias("Total_no_of_unique_coaches"))
no_unique_coaches.show()

+--------------------------+
|Total_no_of_unique_coaches|
+--------------------------+
|                       381|
+--------------------------+



In [0]:
#Identify coaches who coached multiple disciplines.
coaches_who_coaches_multiple_disciplines = df_coaches_schema.groupBy("Name")\
    .agg(count("Discipline").alias("Count_discipline"))\
        .filter(col("Count_discipline")>1)



In [0]:
coaches_who_coaches_multiple_disciplines.show()

+--------------------+----------------+
|                Name|Count_discipline|
+--------------------+----------------+
|GIALLOMBARDO Patr...|               2|
|       FUJIKI Mayuko|               2|
|       SZAUDER Gabor|               2|
|       ELAFANDI Nour|               2|
|REQUENA PERICAS J...|               2|
|   FARINELLI Roberta|               2|
|    MONTICO Loredana|               2|
|            WANG Jie|               2|
|    GUERRERO Rolando|               2|
|   CHEPAK Anastasiya|               2|
| RETEGUI Carlos Jose|               2|
|    TARRES CAMPA Ana|               2|
|  CHERNETSKA Natalia|               2|
+--------------------+----------------+



In [0]:
#Find top 5 countries with most coaches.
countries_most_coaches_five = df_coaches_schema.groupBy("Country")\
    .agg(count("Name").alias("no_of_coaches"))\
        .orderBy(desc("no_of_coaches"))\
        .limit(5)

In [0]:
countries_most_coaches_five.show()

+--------------------+-------------+
|             Country|no_of_coaches|
+--------------------+-------------+
|               Japan|           35|
|               Spain|           28|
|United States of ...|           28|
|           Australia|           22|
|              Canada|           16|
+--------------------+-------------+



In [0]:
 EntriesGender.csv

    Find the discipline with the highest total participation.

    Show total female and male participants across all disciplines.

    Find disciplines with more female than male participants.

    Show disciplines where the total is above 100.

In [0]:
df_entriesgender_schema.columns

Out[166]: ['Discipline', 'Female', 'Male', 'Total']

In [0]:
# Find the discipline with the highest total participation.
discipline_highest_total_participation = df_entriesgender_schema.select(max(col("total")).alias("Highest Total participation"))

In [0]:
discipline_highest_total_participation_groupby = df_entriesgender_schema.groupBy("Discipline")\
    .agg(max(col("total")).alias("Highest Total participation"))\
        .orderBy(desc("Highest Total participation"))\
            .limit(1)

In [0]:
discipline_highest_total_participation.show()
discipline_highest_total_participation_groupby.show()

+---------------------------+
|Highest Total participation|
+---------------------------+
|                       2041|
+---------------------------+

+----------+---------------------------+
|Discipline|Highest Total participation|
+----------+---------------------------+
| Athletics|                       2041|
+----------+---------------------------+



In [0]:
#   Show total female and male participants across all disciplines.
total_male_female = df_entriesgender_schema.groupBy("Discipline")\
    .agg(count("Female"),count("Male"))

In [0]:
total_male_female.show()

+--------------------+-------------+-----------+
|          Discipline|count(Female)|count(Male)|
+--------------------+-------------+-----------+
|              Tennis|            1|          1|
|              Boxing|            1|          1|
|   Marathon Swimming|            1|          1|
|                Golf|            1|          1|
|              Rowing|            1|          1|
|   Baseball/Softball|            1|          1|
|                Judo|            1|          1|
|             Sailing|            1|          1|
|            Swimming|            1|          1|
|Cycling BMX Frees...|            1|          1|
|          Basketball|            1|          1|
|            Handball|            1|          1|
| Rhythmic Gymnastics|            1|          1|
|              Karate|            1|          1|
|           Triathlon|            1|          1|
|           Badminton|            1|          1|
|        Canoe Sprint|            1|          1|
|           Athletic

In [0]:
df_entriesgender_schema.printSchema()


root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



+------------+----------+
|female_count|male_count|
+------------+----------+
|        5432|      5884|
+------------+----------+



In [0]:
#   Show total female and male participants across all disciplines.
total_male_female = df_entriesgender_schema\
    .agg(
        sum(col("Female")).alias("female_count"),
        sum(col("Male")).alias("male_count")
    )

In [0]:
total_male_female.show()

+------------+----------+
|female_count|male_count|
+------------+----------+
|        5432|      5884|
+------------+----------+



In [0]:
#   Find disciplines with more female than male participants.
discipline_female_more_male = df_entriesgender_schema.groupBy("Discipline")\
    .agg(
        sum("Female").alias("female_count")\
        ,sum("Male").alias("male_count"))\
        .filter(col("female_count") > col("male_count"))

In [0]:
discipline_female_more_male.show()

+--------------------+------------+----------+
|          Discipline|female_count|male_count|
+--------------------+------------+----------+
|Cycling BMX Frees...|          10|         9|
| Rhythmic Gymnastics|          96|         0|
|              Diving|          72|        71|
|   Artistic Swimming|         105|         0|
+--------------------+------------+----------+



In [0]:
#   Find disciplines with more male than female participants.
discipline_male_more_female = df_entriesgender_schema.groupBy("Discipline")\
    .agg(
        sum("Female").alias("female_count"),\
        sum("Male").alias("male_count"))\
        .filter(col("male_count") > col("female_count"))

In [0]:
discipline_male_more_female.show()

+-----------------+------------+----------+
|       Discipline|female_count|male_count|
+-----------------+------------+----------+
|           Tennis|          94|        97|
|           Boxing|         102|       187|
|           Rowing|         257|       265|
|Baseball/Softball|          90|       144|
|             Judo|         192|       201|
|         Swimming|         361|       418|
|           Karate|          40|        42|
|        Badminton|          86|        87|
|     Canoe Sprint|         123|       126|
|        Athletics|         969|      1072|
|    Cycling Track|          90|        99|
|       Equestrian|          73|       125|
|     Rugby Sevens|         146|       151|
|     Cycling Road|          70|       131|
|        Wrestling|          96|       193|
|       Water Polo|         122|       146|
|    Weightlifting|          98|        99|
|          Fencing|         107|       108|
|         Football|         264|       344|
+-----------------+------------+

In [0]:
#  Show disciplines where the total is above 100.
discipline_total_above_hundred = df_entriesgender_schema.groupBy("Discipline")\
    .agg(
        sum(col("Female")).alias("female_count")\
        ,sum(col("Male")).alias("male_count")
        )\
            .withColumn("total_count",col("female_count")+  col("male_count"))\
            .filter(col("total_count") > 100)

In [0]:
discipline_total_above_hundred.show()

+-------------------+------------+----------+-----------+
|         Discipline|female_count|male_count|total_count|
+-------------------+------------+----------+-----------+
|             Tennis|          94|        97|        191|
|             Boxing|         102|       187|        289|
|               Golf|          60|        60|        120|
|             Rowing|         257|       265|        522|
|  Baseball/Softball|          90|       144|        234|
|               Judo|         192|       201|        393|
|            Sailing|         175|       175|        350|
|           Swimming|         361|       418|        779|
|         Basketball|         144|       144|        288|
|           Handball|         168|       168|        336|
|          Triathlon|          55|        55|        110|
|          Badminton|          86|        87|        173|
|       Canoe Sprint|         123|       126|        249|
|          Athletics|         969|      1072|       2041|
|      Cycling

In [0]:


🔹 Teams.csv

    Count how many teams participated from each country.

    List all unique disciplines and events in which teams participated.

In [0]:
df_teams_schema.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
#Count how many teams participated from each country.
no_teams_participated_country = df_teams_schema.groupBy("Country")\
    .agg(
        count(col("TeamName")).alias("teams_participated_from_each_country")
    )

In [0]:
no_teams_participated_country.show()90

+--------------------+------------------------------------+
|             Country|teams_participated_from_each_country|
+--------------------+------------------------------------+
|      Chinese Taipei|                                   7|
|Islamic Republic ...|                                   3|
|              Sweden|                                   6|
|   Republic of Korea|                                  19|
|            Malaysia|                                   1|
|           Singapore|                                   1|
|                Fiji|                                   2|
|              Turkey|                                   4|
|             Germany|                                  36|
|              France|                                  33|
|              Greece|                                   6|
|            Slovakia|                                   1|
|    Hong Kong, China|                                   9|
|           Argentina|                  

In [0]:

#To list all unique discipline-event pairs, just use .select().distinct():
unique_disciplines_events = df_teams_schema.select("Discipline","Event")\
    .distinct()

In [0]:
unique_disciplines_events.show()

+-------------------+--------------------+
|         Discipline|               Event|
+-------------------+--------------------+
|         Basketball|               Women|
|  Artistic Swimming|                Duet|
|          Athletics|Men's 4 x 400m Relay|
|   Beach Volleyball|                 Men|
|Artistic Gymnastics|          Men's Team|
|          Athletics|Women's 4 x 400m ...|
|   Beach Volleyball|               Women|
|     3x3 Basketball|                 Men|
|  Baseball/Softball|            Softball|
|          Athletics|Men's 4 x 100m Relay|
|  Baseball/Softball|            Baseball|
|         Basketball|                 Men|
|          Athletics|Women's 4 x 100m ...|
|     3x3 Basketball|               Women|
|  Artistic Swimming|                Team|
|            Archery|          Men's Team|
|Artistic Gymnastics|        Women's Team|
|          Athletics|4 x 400m Relay Mixed|
|            Archery|          Mixed Team|
|            Archery|        Women's Team|
+----------

In [0]:
#List all unique disciplines and events in which teams participated.

disc_event_multiple_teams = df_teams_schema.groupBy("Discipline", "Event") \
    .agg(count("*").alias("team_count")) \
    .filter(col("team_count") > 1)


In [0]:
disc_event_multiple_teams.show()

+-------------------+--------------------+----------+
|         Discipline|               Event|team_count|
+-------------------+--------------------+----------+
|         Basketball|               Women|        12|
|             Hockey|               Women|        12|
|       Table Tennis|          Men's Team|        16|
|  Artistic Swimming|                Duet|        22|
|          Athletics|Men's 4 x 400m Relay|        16|
|   Beach Volleyball|                 Men|        24|
|Artistic Gymnastics|          Men's Team|        12|
|         Volleyball|                 Men|        12|
|       Table Tennis|        Women's Team|        16|
|      Cycling Track|  Men's Team Pursuit|         8|
|       Table Tennis|       Mixed Doubles|        16|
|          Athletics|Women's 4 x 400m ...|        16|
|           Swimming|Women's 4 x 200m ...|        17|
|         Volleyball|               Women|        12|
|   Beach Volleyball|               Women|        24|
|Rhythmic Gymnastics|    Gro

In [0]:
distinct_countries_df_sql = spark.sql("""
                                      SELECT DISTINCT(Country)
                                      FROM
                                      
                                      """)